# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [69]:
using PorousMaterials
using Printf

atomic_masses = read_atomic_masses()
for a in [:Ni1, :Ni2, :Ni3, :Ni4]
    atomic_masses[a] = atomic_masses[:Ni]
end

In [70]:
which_crystal = "NiPyC2"
replication_id = "sc211_" # super cell 2x1x1
qe_run_type = "relax"

# name of the directory containing functionalixed structures
mof_dir_name = which_crystal * "_" * replication_id * qe_run_type

@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "mof_construction",
    "NiPyC2_sc211_relax") 

"/home/cokes/DTRA/mof_construction/NiPyC2_sc211_relax"

In [71]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the :C_aro_R atom on the ring segment of the Fragment
# fragment_list = ["Br", "CH2-CH2-CH3", "CH2-CH3", "CH3", 
#                 "Cl", "C-N", "F", "NH2", "N-NH", 
#                 "O-CH2-CH2-CH3", "O-CH2-CH3", "O-CH3", "OH"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
# substitution_types = ["ortho", "meta"]

In [72]:
# base_name = "NiPyC2_relax"
# xtals = Dict{String, Crystal}()

# for frag in fragment_list
#     for sub_typ in substitution_types
#         xtal_name = base_name * "_" * sub_typ * "_functionalized_" * frag
#         xtals[xtal_name] = Crystal(xtal_name * ".cif")
#     end
# end

In [73]:
# import the functionalized mofs 
crystal = Crystal("NiPyC2_relax_meta_functionalized_NH2.cif")
strip_numbers_from_atom_labels!(crystal)
# relabel Ni atoms
@warn "we labeled Ni's 1-4, but not sure if we need to relabel as Ni1, Ni2 and have two of each."
ni_count = 0
for (i, atom) in enumerate(crystal.atoms.species)
    if atom == :Ni
        ni_count += 1
        crystal.atoms.species[i] = Symbol(@sprintf("Ni%d", ni_count))
    end
end

┌ Warning: we labeled Ni's 1-4, but not sure if we need to relabel as Ni1, Ni2 and have two of each.
└ @ Main In[73]:5


## paths to pseudo-potential libraries

In [81]:
# populate dictionary with names of pseudopotential files
# note: may need to ignore case sensitivity?
# ex: atomic_psp[:Ni] = ["ni_pbesol_v1.4.uspp.F.UPF"]

@warn "fix names"

atom_to_psuedopotential_file = Dict{Symbol, String}()
for atom in [:Ni1, :Ni2, :Ni3, :Ni4]
    atom_to_psuedopotential_file[atom] = "ni.blah"
end
atom_to_psuedopotential_file[:O] = "o.blah"
atom_to_psuedopotential_file[:C] = "c.blah"
atom_to_psuedopotential_file[:N] = "N.blah"
atom_to_psuedopotential_file[:H] = "H.blah"

┌ Warning: fix names
└ @ Main In[81]:5


"H.blah"

## default QE structural relaxation params

In [82]:
"""
    input_params = qe_input_params(crystal; kwargs)

create a dictionary consisting of input arguments for QE structural relaxation
needed for the script and set default values
then populate with values specific to each material
link with full list and description of input params
https://www.quantum-espresso.org/Doc/INPUT_PW.html#__top__

WARNING: tailored to Nickel
"""
function qe_structural_opt_input_params(crystal::Crystal; kwargs...)
    @warn "tailored to NiPyC and their analogues"
    
    input_params = Dict{Symbol, Any}()
    # &control params
    input_params[:calculation] = "relax" 
    input_params[:prefix] = split(crystal.name, ".")[1]
    input_params[:pseudo_dir] = joinpath(pwd(), "structural_relaxation", "pseudo")
    input_params[:outdir] = joinpath(pwd(), "structural_relaxation")
    input_params[:nstep] = 200

    # &SYSTEM params
    input_params[:ibrav] = 0
    input_params[:nat] = crystal.atoms.n
    input_params[:ntyp] = length(unique(crystal.atoms.species))
    input_params[:tot_charge] = 0.0
    input_params[:ecutwfc] = 100.0
    input_params[:vdw_corr] = "grimme-d2"
    input_params[:occupations] = "smearing"
    input_params[:degauss] = 0.2
    input_params[:smearing] = "mv"
    input_params[:nspin] = 2
    input_params[:sub_lat_species_index] = ["1", "2"]
    input_params[:starting_magnetization] = [0.25, 0.25]
    input_params[:tot_magnetization] = 8 # check calculation

    # &ELECTRON params
    input_params[:electron_maxstep] = 90
    input_params[:mixing_mode] = "local-TF"
    input_params[:diagonalization] = "david"

    # &IONS params
    input_params[:ion_dynamics] = "bfgs"

    # &CELL params
    input_params[:cell_dynamics] = "bfgs"
    input_params[:cell_dofree] = "xyz"

    # K_POINTS params
    input_params[:k_points] = ["{gamma}"]
    
    # over-ride default params with ones passed in kwargs
    for (key, value) in kwargs
        input_params[key] = value
    end
    return input_params
end

input_params = qe_structural_opt_input_params(crystal)

┌ Warning: tailored to NiPyC and their analogues
└ @ Main In[82]:13


Dict{Symbol,Any} with 25 entries:
  :starting_magnetization => [0.25, 0.25]
  :ecutwfc                => 100.0
  :vdw_corr               => "grimme-d2"
  :occupations            => "smearing"
  :diagonalization        => "david"
  :prefix                 => "NiPyC2_relax_meta_functionalized_NH2"
  :nspin                  => 2
  :outdir                 => "/home/cokes/DTRA/structural_relaxation"
  :pseudo_dir             => "/home/cokes/DTRA/structural_relaxation/pseudo"
  :ntyp                   => 8
  :smearing               => "mv"
  :ibrav                  => 0
  :degauss                => 0.2
  :mixing_mode            => "local-TF"
  :k_points               => ["{gamma}"]
  :sub_lat_species_index  => ["1", "2"]
  :tot_charge             => 0.0
  :calculation            => "relax"
  :ion_dynamics           => "bfgs"
  :cell_dynamics          => "bfgs"
  :nstep                  => 200
  :nat                    => 124
  :cell_dofree            => "xyz"
  :electron_maxstep       => 90


In [84]:
function check_psuedopotential_coverage(crystal::Crystal)
    for atom in unique(crystal.atoms.species)
        if ! (atom in keys(atom_to_psuedopotential_file))
            error(@sprintf("atom %s not in `atom_to_psuedopotential_file`", atom))
        end
    end
end

check_psuedopotential_coverage(crystal)

In [78]:
function generate_qe_structural_opt_input_file(crystal::Crystal, input_params::Dict{Symbol, Any})
    # check pseudo-potential coverage
    check_psuedopotential_coverage(crystal)
    
    qe_input_filename = "pw." * split(crystal.name, ".cif")[1] * ".in"
    qe_input_file = open(qe_input_filename, "w")

    # write &Control
    @printf(qe_input_file,
        """
        &CONTROL
           calculation = '%s',
           prefix = '%s',
           pseudo_dir = '%s',
           outdir = '%s'
           nstep = %.0f,
        /
        """,
        input_params[:calculation], input_params[:prefix],
        input_params[:pseudo_dir],
        input_params[:outdir], input_params[:nstep]
    )

    # write &SYSTEM
    @printf(qe_input_file,
        """
        &SYSTEM
           ibrav = %.0f,
           nat = %.0f,
           ntyp = %0.f,
           tot_charge = %f,
           ecutwfc = %f,
           vdw_corr = '%s',
           occupations = '%s',
           degauss = %f,
           smearing = '%s',
           nspin = %.0f ,
           starting_magnetization(%s) = %f,
           starting_magnetization(%s) = %f,
           tot_magnetization = %f
        /
        """, 
        input_params[:ibrav], input_params[:nat],
        input_params[:ntyp], input_params[:tot_charge],
        input_params[:ecutwfc], input_params[:vdw_corr],
        input_params[:occupations], input_params[:degauss],
        input_params[:smearing], input_params[:nspin],
        input_params[:sub_lat_species_index][1],
        input_params[:starting_magnetization][1],
        input_params[:sub_lat_species_index][2],
        input_params[:starting_magnetization][2],
        input_params[:tot_magnetization]
    )

    # write &ELECTRONS
    @printf(qe_input_file,
        """
        &ELECTRONS
        electron_maxstep = %0.f,
        mixing_mode = '%s',
        diagonalization = '%s',
        /
        """,
        input_params[:electron_maxstep], input_params[:mixing_mode], 
        input_params[:diagonalization]
    )

    # write &IONS
    @printf(qe_input_file,
        """
        &IONS
        ion_dynamics = '%s',
        /
        """,
        input_params[:ion_dynamics]
    )


    # write &CELL
    @printf(qe_input_file,
        """
        &CELL
        cell_dynamics = '%s'
        cell_dofree = '%s'
        /
        """,
        input_params[:cell_dynamics], input_params[:cell_dofree]
    )

    # write ATOMIC_SPECIES
    @printf(qe_input_file,
        """
        ATOMIC_SPECIES
        """
    )
    
    @warn "address Ni vs Ni2"
    for atom in unique(crystal.atoms.species)
        @printf(qe_input_file,
            """
            %s    %f    %s
            """,
            atom, atomic_masses[atom], atom_to_psuedopotential_file[atom]
        )
    end

    # write K_POINTS
    @printf(qe_input_file,
        """
        K_POINTS %s
        """,
        input_params[:k_points]
    )


    # write CELL_PARAMETERS
    @printf(qe_input_file,
        """
        CELL_PARAMETERS "{angstrom}"
        """
    )
    for d = 1:3
        @printf(qe_input_file,
            "     %f   %f   %f\n",
            crystal.box.f_to_c[1, d], crystal.box.f_to_c[2, d], crystal.box.f_to_c[3, d]
        )
    end

    # write ATOMIC_POSITIONS
    #symbol #x #y #z
    @printf(qe_input_file, "ATOMIC_POSITIONS {crystal}\n")


    for a = 1:crystal.atoms.n
        @printf(qe_input_file, 
            "%s    %f  %f  %f\n",
            crystal.atoms.species[a], 
            crystal.atoms.coords.xf[1, a], 
            crystal.atoms.coords.xf[2, a], 
            crystal.atoms.coords.xf[3, a]
        )
    end
    close(qe_input_file)
    return qe_input_filename
end

input_params = qe_structural_opt_input_params(crystal)
qe_input_filename = generate_qe_structural_opt_input_file(crystal, input_params)

┌ Warning: tailored to NiPyC and their analogues
└ @ Main In[75]:13
┌ Info: :) we hv psuedopotential files for all atoms of NiPyC2_relax_meta_functionalized_NH2.cif
└ @ Main In[76]:7
┌ Warning: address Ni vs Ni2
└ @ Main In[78]:97


"pw.NiPyC2_relax_meta_functionalized_NH2.in"

In [79]:
sum(crystal.atoms.species .== :Ni)

0

In [80]:
run(`cat $qe_input_filename`)

&CONTROL
   calculation = 'relax',
   prefix = 'NiPyC2_relax_meta_functionalized_NH2',
   pseudo_dir = '/home/cokes/DTRA/structural_relaxation/pseudo',
   outdir = '/home/cokes/DTRA/structural_relaxation'
   nstep = 200,
/
&SYSTEM
   ibrav = 0,
   nat = 124,
   ntyp = 8.000000,
   tot_charge = 0.000000,
   ecutwfc = 100.000000,
   vdw_corr = 'grimme-d2',
   occupations = 'smearing',
   degauss = 0.200000,
   smearing = 'mv',
   nspin = 2 ,
   starting_magnetization(1) = 0.250000,
   starting_magnetization(2) = 0.250000,
   tot_magnetization = 8.000000
/
&ELECTRONS
electron_maxstep = 90.000000,
mixing_mode = 'local-TF',
diagonalization = 'david',
/
&IONS
ion_dynamics = 'bfgs',
/
&CELL
cell_dynamics = 'bfgs'
cell_dofree = 'xyz'
/
ATOMIC_SPECIES
Ni1    58.693400    ni.blah
Ni2    58.693400    ni.blah
O    15.999400    o.blah
N    14.006700    N.blah
C    12.010700    c.blah
H    1.007940    H.blah
Ni3    58.693400    ni.blah
Ni4    58.693400    ni.blah
K_POINTS ["{gamma}"]
CELL_PARAMETERS

Process(`cat pw.NiPyC2_relax_meta_functionalized_NH2.in`, ProcessExited(0))